In [1]:
import base64
import pandas as pd
import requests as rq
import json
from datetime import date

In [ ]:

# Preparar Token
def get_outh_token():
    Apikey = '0gs9fabdhesbisczsyx2pjhvvxq8hg9z'
    Secret = 'SoWh9TefKizy'

    message = f"{Apikey}:{Secret}"
    auth = "Basic " + base64.b64encode(message.encode("ascii")).decode("ascii")

    headers_dic = {
        "Authorization": auth,
        "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"
    }
    data_dic = {
        "grant_type": "client_credentials",
        "scope": "read"
    }

    r = rq.post("https://api.idealista.com/oauth/token", headers=headers_dic, data=data_dic)
    if r.status_code != 200:
        raise Exception(f"Error al obtener token: {r.status_code} - {r.text}")

    return r.json()['access_token']

# Buscar infromación selecionada
def search_api(pagination=1):
    token = get_outh_token()
    headers = {'Authorization': 'Bearer ' + token}

    data = {
        "operation": "sale",
        "propertyType": "homes",
        "center": "37.365733231921354,-6.007946780316465",
        "distance": 7000,  # en metros (7 km)
        "maxItems": 50,
        "numPage": pagination,
        "language": "es",
        "country": "es"
    }

    r = rq.post("https://api.idealista.com/3.5/es/search", headers=headers, data=data)

    if r.status_code != 200:
        raise Exception(f"Error en búsqueda: {r.status_code} - {r.text}")

    return r.json()

# Recorrer 45 paginas
all_houses = []
total_pages = 45  

for page in range(1, total_pages + 1):
    print(f"Descargando página {page}/{total_pages}...")
    data = search_api(page)

    houses = data.get("elementList", [])
    all_houses.extend(houses)

    # Parar por si no hay mas
    if not houses:
        break

print(f"✅ Total casas descargadas: {len(all_houses)}")

# Convertir en DataFrame
df = pd.DataFrame(all_houses)

columnas_interes = [
    "propertyCode", "price", "propertyType", "operation",
    "rooms", "bathrooms", "size", "address", "province",
    "municipality", "district", "neighborhood", "latitude",
    "longitude", "distance", "exterior", "floor", "parkingSpace", "priceByArea", "detailedType"
]
df = df[columnas_interes]

# Gardar en csv
nombre_csv = f"ventas_sevilla_{date.today()}.csv"
df.to_csv(nombre_csv, index=False, encoding="utf-8-sig")

print(f"📂 CSV guardado como: {nombre_csv}")

Descargando página 1/45...
Descargando página 2/45...
Descargando página 3/45...
Descargando página 4/45...
Descargando página 5/45...
Descargando página 6/45...
Descargando página 7/45...
Descargando página 8/45...
Descargando página 9/45...
Descargando página 10/45...
Descargando página 11/45...
Descargando página 12/45...
Descargando página 13/45...
Descargando página 14/45...
Descargando página 15/45...
Descargando página 16/45...
Descargando página 17/45...
Descargando página 18/45...
Descargando página 19/45...
Descargando página 20/45...
Descargando página 21/45...
Descargando página 22/45...
Descargando página 23/45...
Descargando página 24/45...
Descargando página 25/45...
Descargando página 26/45...
Descargando página 27/45...
Descargando página 28/45...
Descargando página 29/45...
Descargando página 30/45...
Descargando página 31/45...
Descargando página 32/45...
Descargando página 33/45...
Descargando página 34/45...
Descargando página 35/45...
Descargando página 36/45...
D

In [5]:
ruta = 'ventas_sevilla_2025-07-23.csv'
df=pd.read_csv(ruta)

In [7]:
df["municipality"].unique()

array(['Sevilla', 'Camas', 'Bormujos', 'San Juan de Aznalfarache',
       'Mairena del Aljarafe', 'Tomares', 'Montequinto',
       'Castilleja de Guzman', 'Castilleja de la Cuesta', 'Gelves',
       'Gines', 'Palomares del Río', 'Dos Hermanas',
       'Valencina de la Concepcion'], dtype=object)

In [8]:
df["district"].unique()

array(['Centro', 'Nervión', 'Los Remedios',
       'Santa Justa - Miraflores - Cruz Roja', nan, 'Triana',
       'Prado de San Sebastián - Felipe II - Bueno Monreal',
       'La Palmera - Los Bermejales', 'Bellavista - Jardines de Hércules',
       'El Zaudín - Club de Golf', 'Barrio Alto', 'Bulevard norte',
       'Macarena', 'Nuevo Bulevar', 'Santa Clara', 'Las Almenas',
       'Ciudad Aljarafe', 'Cerro Amate', 'Simon Verde', 'Santa Eufemia',
       'Barrio Bajo', 'Condequinto', 'Pino Montano', 'Zona Universitaria',
       'Sevilla este', 'San Jerónimo', 'Lepanto', 'Aljamar',
       'Zona Avenida de Europa', 'Arco norte - Avda España',
       'Zona Avda. Juan Diego', 'San Pablo', 'Cavaleri',
       'Hacienda la Cartuja', 'Ciudad Expo'], dtype=object)

In [5]:
df["propertyType"].unique()

array(['flat', 'chalet', 'penthouse', 'studio', 'duplex', 'countryHouse'],
      dtype=object)

In [6]:
df['parkingSpace'].unique()

array(["{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}",
       nan,
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 60000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 30000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 40000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 35000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 20000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 25000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 43000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': False, 'parkingSpacePrice': 70000.0}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice'

In [7]:
df["detailedType"].unique()

array(["{'typology': 'flat'}",
       "{'typology': 'chalet', 'subTypology': 'independantHouse'}",
       "{'typology': 'flat', 'subTypology': 'penthouse'}",
       "{'typology': 'flat', 'subTypology': 'studio'}",
       "{'typology': 'chalet', 'subTypology': 'terracedHouse'}",
       "{'typology': 'flat', 'subTypology': 'duplex'}",
       "{'typology': 'chalet'}",
       "{'typology': 'chalet', 'subTypology': 'semidetachedHouse'}",
       "{'typology': 'countryHouse', 'subTypology': 'casaDePueblo'}"],
      dtype=object)

In [10]:
df["propertyType"].value_counts()


propertyType
flat            1574
chalet           507
duplex            82
penthouse         71
studio            15
countryHouse       1
Name: count, dtype: int64

"rooms", "bathrooms"

In [11]:
df["bathrooms"].value_counts()

bathrooms
2     969
1     773
3     334
4      92
5      37
6      20
7      13
8       5
13      2
0       2
11      2
24      1
Name: count, dtype: int64

In [12]:
df["rooms"].value_counts()

rooms
3     924
4     475
2     394
5     160
1     148
6      69
7      28
0      18
8      17
9       5
10      3
12      3
11      3
15      1
13      1
22      1
Name: count, dtype: int64

floor

In [13]:
df["floor"].value_counts()

floor
1     429
2     312
bj    282
3     231
4     165
5      91
6      45
7      28
8      26
en     19
9      14
11      5
10      4
12      3
13      2
ss      1
-1      1
Name: count, dtype: int64